# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

# Section 1:Business Understaning

The starbucks data give a situation about Sale Promotion with cupon. It give the customers a discount of BOGO(buy one get one free), which will prompt the sale if the customer use it. Customer get discount, starbuck get sale promotion and build customer potential consumption pattern.

For accuracy we should find out how to use the cupon:
1. Is it benifit for starbucks
2. if it worthy enough, who should we give the cupon.

# Section 2:Data Understaning

1. Reading in data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import json
import re
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

2. we will take a look at the data first.

portfolio seems no need for cleaning.

2. Data Cleaning

In [2]:
#profile['income_na'] = profile['income'].isna()
#profile['income'].fillna(0, inplace = True)

In [3]:
print(profile.shape)
profile.head(10)

(17000, 5)


,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN
5,M,68,e2127556f4f64592b11af22de27a7932,20180426,70000.0
6,None,118,8ec6ce2a7e7949b1bf142def7d0e0586,20170925,NaN
7,None,118,68617ca6246f4fbc85e91a2a49552598,20171002,NaN
8,M,65,389bc3fa690240e798340f5a15918d5c,20180209,53000.0
9,None,118,8974fc5686fe429db53ddde067b88302,20161122,NaN


In [132]:
user_id_map = pd.DataFrame(
        {'person':profile['id'],
         'person_id': range(len(profile['id']))
        })
user_id_map.head()
profile_new = profile.merge(user_id_map,how = 'left', left_on = 'id', right_on = 'person')
profile_new.drop(['id', 'person'], axis = 1, inplace = True)
profile_new.loc[profile_new['age'] >= 100 , 'age'] = np.nan
profile_new.loc[profile_new['gender'] == 'M' , 'gender'] = 1.0
profile_new.loc[profile_new['gender'] == 'F' , 'gender'] = 0.0
profile_new.loc[profile_new['gender'] == 'O' , 'gender'] = 0.5
profile_new['gender'].fillna(np.nan, inplace = True)
profile_new['gender'].astype(float)
profile_new.head(10)

,gender,age,became_member_on,income,person_id
0,NaN,NaN,20170212,NaN,0
1,0.0,55.0,20170715,112000.0,1
2,NaN,NaN,20180712,NaN,2
3,0.0,75.0,20170509,100000.0,3
4,NaN,NaN,20170804,NaN,4
5,1.0,68.0,20180426,70000.0,5
6,NaN,NaN,20170925,NaN,6
7,NaN,NaN,20171002,NaN,7
8,1.0,65.0,20180209,53000.0,8
9,NaN,NaN,20161122,NaN,9


In [135]:
profile_new['gender'].unique()

array([nan, 0. , 1. , 0.5])

there are a lot of customers at age 118.Those are the people who do not want to give the age informations. We will consider that as a NaN.

In [136]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


In [137]:
portfolio_id = portfolio['id'].unique()
offer_id = np.arange(len(portfolio_id))
offer_id_map = pd.DataFrame(
    {'portfolio_id': portfolio_id,
     'offer_id': offer_id
    })

In [138]:
offer_id_map['offer_id'].dtype

dtype('int64')

In [139]:
def clean_portfolio(portfolio = portfolio, offer_id_map = offer_id_map):
    """
    clean the portfolio data, change "ID" into "offer_id" according to offer_id_map
    split channels into columns
    
    
    """
    port_clean = portfolio.copy()
    channel_list = []
    for channel in portfolio['channels']:
        channel_list.extend(channel)
    channel_set = set(channel_list)
    
    # convert channels into dummies
    for channel in channel_set:
        port_clean['channel_' + channel] = port_clean['channels'].apply(lambda x: x.count(channel)>0)
        
    # change id according to offer_id_map
    
    port_clean['offer_id'] = port_clean['id'].apply(lambda x:
                            int(offer_id_map[offer_id_map['portfolio_id'] == x]['offer_id'].values))
        
    
    port_clean.drop(['channels','id'], axis = 1, inplace = True)
    return port_clean

clean_portfolio()

,reward,difficulty,duration,offer_type,channel_social,channel_web,channel_mobile,channel_email,offer_id
0,10,10,7,bogo,True,False,True,True,0
1,10,10,5,bogo,True,True,True,True,1
2,0,0,4,informational,False,True,True,True,2
3,5,5,7,bogo,False,True,True,True,3
4,5,20,10,discount,False,True,False,True,4
5,3,7,7,discount,True,True,True,True,5
6,2,10,10,discount,True,True,True,True,6
7,0,0,3,informational,True,False,True,True,7
8,5,5,5,bogo,True,True,True,True,8
9,2,10,7,discount,False,True,True,True,9


In [140]:
(transcript['person'].shape)

(306534,)

In [141]:
transcript['person'].nunique()

17000

In [142]:
transcript[transcript['person'] == '78afa995795e4d85b5d9ceeca43f5fef']

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
15561,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},6
47582,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 19.89},132
47583,78afa995795e4d85b5d9ceeca43f5fef,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,132
49502,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 17.78},144
53176,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},168
85291,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},216
87134,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 19.67},222
92104,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 29.72},240
141566,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 23.93},378


In [143]:
portfolio_clean = clean_portfolio()

In [144]:
def clean_transcript(transcript = transcript, portfolio = clean_portfolio, offer_id_map = offer_id_map, user_id_map = user_id_map):
    """
    clean the transcript data, use person_id, offer_id instead of hash number.
    
    Parameters
    ------------
    
    
    Returns
    ------------
    
    
    """
    
    # simplify person to person_id
    transcript_clean = transcript.copy()    

    transcript_clean = transcript_clean.merge(user_id_map, on = 'person')    
    print(transcript_clean.event.unique())
    # 
    
    transcript_clean['portfolio_id'] = transcript_clean['value'].apply(
        lambda x : x.get('offer_id', np.nan) if (x.get('offer id', np.nan) is np.nan) else x.get('offer id', np.nan)
    )


    transcript_clean['amount'] = transcript_clean['value'].apply(lambda x :x.get('amount', np.nan))

    transcript_clean['reward'] = transcript_clean['value'].apply(lambda x :x.get('reward', np.nan))
    transcript_clean = transcript_clean.merge(offer_id_map,how = 'left', on = 'portfolio_id')
    transcript_clean['offer_id'] = transcript_clean['offer_id'].astype('Int64')
    transcript_clean.drop(['person','value', 'portfolio_id'], axis = 1, inplace = True)

    return transcript_clean

clean_trans = clean_transcript()
clean_trans.head()

['offer received' 'offer viewed' 'transaction' 'offer completed']


,event,time,person_id,amount,reward,offer_id
0,offer received,0,3,NaN,NaN,3
1,offer viewed,6,3,NaN,NaN,3
2,transaction,132,3,19.89,NaN,<NA>
3,offer completed,132,3,NaN,5.0,3
4,transaction,144,3,17.78,NaN,<NA>


In [158]:
def create_offer_received_completed_matrix(data, profile = profile_new ):
    event_offer_cp = data['event'] == 'offer completed'
    event_offer_re = data['event'] == 'offer received'
    event_offer_vi = data['event'] == 'offer viewed' 
    event_offer_ta = data['event'] == 'offer transaction' 
    
    trans_offer_tag = data[data['event'].isin(['offer completed', 'transaction'])]
    time_duplicated = trans_offer_tag.duplicated(['time', 'person_id'], keep = False)
    
    # all transaction 

    # trans with out offer
    trans_without_offer_df = trans_offer_tag[~ time_duplicated]
    # trans with offer 
    trans_with_offer_df = trans_offer_tag[time_duplicated]

    
    
    # count completed offer
    offer_completed_count = data[event_offer_cp].groupby(['person_id', 'offer_id'])['person_id'].count().unstack(1)

    # offer_completed_count = offer_completed_count.add_suffix('_completed')
    # count received offer
    offer_received_count = data[event_offer_re].groupby(['person_id', 'offer_id'])['person_id'].count().unstack(1)


    #offer_received_count = offer_received_count.add_suffix('_received')    

    #    offer_received_count = data[event_offer_re].groupby(['person_id', 'offer_id'])['person_id'].count().unstack(1)
    # offer_received_count = offer_received_count.add_suffix('_received')    
    reward_ = data[event_offer_cp].groupby(['person_id','time'])['reward'].sum()
    reward_nozero = reward_[reward_!=0]
    mean_reward = reward_nozero.groupby(level = [0]).mean()

    # feature_matrix = offer_completed_count.merge(offer_received_count, on = 'person_id')
    row = offer_received_count.index
    column = offer_received_count.columns
    
    feature_matrix = pd.DataFrame(np.nan, index = row, columns = column)
    for i in row:
        for j in column:
            try:
                if pd.isna(offer_received_count.loc[i,j]):
                    feature_matrix.loc[i,j] = np.nan
                elif pd.isna(offer_completed_count.loc[i,j]):
                    feature_matrix.loc[i,j] = 0
                else:
                    feature_matrix.loc[i,j] = offer_completed_count.loc[i,j] / offer_received_count.loc[i,j]
            except:
                feature_matrix.loc[i,j] = 0
                continue
    # feature_matrix = offer_completed_count / offer_received_count
    

    
    # get mean amount without offer
    trans_mean_without_offer = trans_without_offer_df.groupby(['person_id'])['amount'].mean()
    trans_mean_without_offer.rename('mean_trans_without_offer', inplace = True)
    # trans_count_without_offer = trans_without_offer_df.groupby(['person_id'])['amount'].count()
    # trans_count_without_offer.rename('count_trans_without_offer')
    # get mean amount with offer
    trans_mean_with_offer = trans_with_offer_df.groupby(['person_id'])['amount'].mean()
    
    trans_mean_with_offer.rename("mean_trans_with_offer", inplace = True)
#    trans_count_with_offer = trans_with_offer_df.groupby(['person_id'])['amount'].count()
#    trans_count_with_offer.rename('count_trans_with_offer')    
    feature_matrix = feature_matrix.merge(trans_mean_without_offer, how = 'left', on = 'person_id')
    feature_matrix = feature_matrix.merge(trans_mean_with_offer,how = 'left', on = 'person_id')
    feature_matrix = feature_matrix.merge(mean_reward,how = 'left', on = 'person_id')
    feature_matrix = feature_matrix.merge(profile_new,how = 'left', on = 'person_id')
    feature_matrix.drop(['became_member_on', 'person_id'],axis = 1, inplace = True)
    feature_matrix['reward'] = feature_matrix['reward']/10
    feature_matrix['income'] = feature_matrix['income']/100000
    feature_matrix['age'] = feature_matrix['age']/100
    feature_matrix['mean_trans_without_offer'] = feature_matrix['mean_trans_without_offer']/10
    feature_matrix['mean_trans_with_offer'] = feature_matrix['mean_trans_with_offer']/10
    return feature_matrix
#trans_feature = create_matrix(clean_trans)
trans_df = create_offer_received_completed_matrix(clean_trans)

trans_df.head()

,0,1,2,3,4,5,6,7,8,9,mean_trans_without_offer,mean_trans_with_offer,reward,gender,age,income
0,NaN,NaN,NaN,NaN,0.0,1.0,0.5,NaN,NaN,0.0,0.228500,0.2120,0.5,NaN,NaN,NaN
1,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.689500,2.3220,0.5,0.0,0.55,1.12
2,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.238333,NaN,NaN,NaN,NaN,NaN
3,1.0,NaN,NaN,1.0,NaN,NaN,NaN,0.0,1.0,NaN,2.353200,2.0805,1.0,0.0,0.75,1.00
4,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.155000,NaN,NaN,NaN,NaN,NaN


In [159]:
trans_df.describe()

,0,1,2,3,4,5,6,7,8,9,mean_trans_without_offer,mean_trans_with_offer,reward,gender,age,income
count,6374.000000,6330.000000,6331.0,6355.000000,6374.000000,6325.000000,6332.000000,6320.0,6262.000000,6285.000000,16150.000000,12774.000000,12774.000000,14820.000000,14803.000000,14820.000000
mean,0.480454,0.436862,0.0,0.568725,0.447396,0.676864,0.698347,0.0,0.568958,0.527513,1.272283,1.898487,0.531398,0.579555,0.543364,0.654069
std,0.490657,0.486484,0.0,0.481846,0.487389,0.456098,0.449856,0.0,0.480934,0.486079,1.711185,2.700130,0.259452,0.490011,0.173234,0.215981
min,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.005000,0.023000,0.200000,0.000000,0.180000,0.300000
25%,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.278500,0.940250,0.350000,0.000000,0.420000,0.490000
50%,0.000000,0.000000,0.0,1.000000,0.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.018550,1.623000,0.500000,1.000000,0.550000,0.640000
75%,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.932917,2.272000,0.666667,1.000000,0.660000,0.800000
max,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,55.066500,101.573000,2.500000,1.000000,0.990000,1.200000


In [160]:
trans_df.columns

Index([                         0,                          1,
                                2,                          3,
                                4,                          5,
                                6,                          7,
                                8,                          9,
       'mean_trans_without_offer',    'mean_trans_with_offer',
                         'reward',                   'gender',
                            'age',                   'income'],
      dtype='object')

In [161]:
trans_mat_df = trans_df.drop([2,7],axis = 1)
trans_mat = trans_mat_df.to_numpy()
trans_mat.shape

(16994, 14)

In [149]:
def FunkSVD(trans_mat , latent_features=14, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = trans_mat.shape[0] # number of rows in the matrix
    n_feature = trans_mat.shape[1] # number of movies in the matrix
    num_ratings = n_feature*n_users - np.isnan(trans_mat).sum() # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    feature_mat = np.random.rand(latent_features, n_feature) # movie matrix filled with random values of shape latent x movies

    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for i in range(iters):
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for row in range(n_users):
            for column in range(n_feature):
                # if the rating exists
                if np.isnan(trans_mat[row,column]) ==False :
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    error_mat = trans_mat[row,column] - np.dot(user_mat[row,:],feature_mat[:,column])
                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += error_mat**2
                    # update the values in each matrix in the direction of the gradient
                    user_mat[row,:] +=  learning_rate*2*error_mat*feature_mat[:,column]
                    feature_mat[:,column] += + learning_rate*2*error_mat*user_mat[row,:]
                    # print results for iteration
        print('SSE for {} iter: {}'.format(i+1, sse_accum))
    return user_mat, feature_mat 

Funk SVD


In [51]:
user_mat, feature_mat = FunkSVD(trans_mat, learning_rate = 0.01)

Optimization Statistics
Iterations | Mean Squared Error 
SSE for 1 iter: 17978.68063477033
SSE for 2 iter: 16254.572706643057
SSE for 3 iter: 16117.397267414057
SSE for 4 iter: 15980.191937159685
SSE for 5 iter: 15842.584365544535
SSE for 6 iter: 15704.135346828656
SSE for 7 iter: 15564.330352052946
SSE for 8 iter: 15422.570363923338
SSE for 9 iter: 15278.163315918435
SSE for 10 iter: 15130.317075721689
SSE for 11 iter: 14978.135284359661
SSE for 12 iter: 14820.617790955199
SSE for 13 iter: 14656.667871153652
SSE for 14 iter: 14485.108811167096
SSE for 15 iter: 14304.712657657952
SSE for 16 iter: 14114.243803231291
SSE for 17 iter: 13912.519384664118
SSE for 18 iter: 13698.487002045362
SSE for 19 iter: 13471.317882968428
SSE for 20 iter: 13230.510363629257
SSE for 21 iter: 12975.994788196607
SSE for 22 iter: 12708.227362080253
SSE for 23 iter: 12428.258175749406
SSE for 24 iter: 12137.758671203152
SSE for 25 iter: 11838.997065044547
SSE for 26 iter: 11534.756723126426
SSE for 27 iter: 

In [58]:
user_mat2, feature_mat2 = FunkSVD(trans_mat, learning_rate = 0.015, iters = 250)

Optimization Statistics
Iterations | Mean Squared Error 
SSE for 1 iter: 18112.108319247873
SSE for 2 iter: 16594.0835632044
SSE for 3 iter: 16369.43071406232
SSE for 4 iter: 16141.91057880068
SSE for 5 iter: 15909.500038136552
SSE for 6 iter: 15669.723927582387
SSE for 7 iter: 15419.629906730617
SSE for 8 iter: 15155.814950307311
SSE for 9 iter: 14874.535890381698
SSE for 10 iter: 14571.939031754873
SSE for 11 iter: 14244.435416431785
SSE for 12 iter: 13889.222467538388
SSE for 13 iter: 13504.9075494224
SSE for 14 iter: 13092.132576472135
SSE for 15 iter: 12654.051656393329
SSE for 16 iter: 12196.502201794638
SSE for 17 iter: 11727.751612744685
SSE for 18 iter: 11257.793016384878
SSE for 19 iter: 10797.280607458693
SSE for 20 iter: 10356.303262280237
SSE for 21 iter: 9943.256507238793
SSE for 22 iter: 9564.055078621459
SSE for 23 iter: 9221.825415502408
SSE for 24 iter: 8917.06917655496
SSE for 25 iter: 8648.163626083631
SSE for 26 iter: 8412.012722569976
SSE for 27 iter: 8204.6856675

In [59]:
user_mat2, feature_mat2 = FunkSVD(trans_mat, learning_rate = 0.017, iters = 250)

Optimization Statistics
Iterations | Mean Squared Error 
SSE for 1 iter: 18152.85001680402
SSE for 2 iter: 16782.769906550628
SSE for 3 iter: 16520.582925327224
SSE for 4 iter: 16254.887544379922
SSE for 5 iter: 15982.964381702677
SSE for 6 iter: 15701.498247061765
SSE for 7 iter: 15406.58918036479
SSE for 8 iter: 15093.877887350329
SSE for 9 iter: 14758.847149749856
SSE for 10 iter: 14397.351418745115
SSE for 11 iter: 14006.38011176103
SSE for 12 iter: 13584.96606525153
SSE for 13 iter: 13135.028224167425
SSE for 14 iter: 12661.849090717296
SSE for 15 iter: 12173.915398500963
SSE for 16 iter: 11682.03322101736
SSE for 17 iter: 11197.902518808281
SSE for 18 iter: 10732.55311794525
SSE for 19 iter: 10295.071583753728
SSE for 20 iter: 9891.875817057511
SSE for 21 iter: 9526.549180649134
SSE for 22 iter: 9200.082942520818
SSE for 23 iter: 8911.344138553104
SSE for 24 iter: 8657.62894239736
SSE for 25 iter: 8435.213717020944
SSE for 26 iter: 8239.851992401253
SSE for 27 iter: 8067.18392280

In [150]:
trans_mat

array([[ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan, 1.  , ..., 0.  , 0.55, 1.12],
       [ nan,  nan, 0.  , ...,  nan,  nan,  nan],
       ...,
       [ nan,  nan,  nan, ..., 1.  , 0.49, 0.73],
       [1.  , 1.  , 1.  , ..., 0.  , 0.83, 0.5 ],
       [ nan,  nan,  nan, ..., 0.  , 0.62, 0.82]])

In [169]:
user_mat3, feature_mat3 = FunkSVD(trans_mat, learning_rate = 0.001, iters = 500)

Optimization Statistics
Iterations | Mean Squared Error 
SSE for 1 iter: 172730.5385249889
SSE for 2 iter: 160381.80127234408
SSE for 3 iter: 159576.9644845807
SSE for 4 iter: 158923.04905612103
SSE for 5 iter: 158253.93394181502
SSE for 6 iter: 157538.19810835007
SSE for 7 iter: 156753.87470696575
SSE for 8 iter: 155873.18175368352
SSE for 9 iter: 154858.45148995347
SSE for 10 iter: 153658.87210720088
SSE for 11 iter: 152206.95451843712
SSE for 12 iter: 150415.03647854432
SSE for 13 iter: 148172.85147527745
SSE for 14 iter: 145348.15841006837
SSE for 15 iter: 141793.71467073858
SSE for 16 iter: 137365.0243124319
SSE for 17 iter: 131952.94220269533
SSE for 18 iter: 125530.91929082095
SSE for 19 iter: 118206.3326357676
SSE for 20 iter: 110251.5777708813
SSE for 21 iter: 102085.17986444679
SSE for 22 iter: 94191.08437121054
SSE for 23 iter: 87002.8338611153
SSE for 24 iter: 80807.33797405622
SSE for 25 iter: 75710.51458267463
SSE for 26 iter: 71665.33940394035
SSE for 27 iter: 68531.1124

In [170]:
pred = np.dot(user_mat3,feature_mat3)

In [171]:
pred_df = pd.DataFrame(pred, index = trans_mat_df.index, columns = trans_mat_df.columns )
pred_df.head()

,0,1,3,4,5,6,8,9,mean_trans_without_offer,mean_trans_with_offer,reward,gender,age,income
0,0.226803,0.289082,0.484476,0.293518,0.571424,0.561434,0.576972,0.398393,0.197523,0.205244,0.396920,0.652485,0.515165,0.585959
1,0.829749,0.723070,0.945269,0.632886,1.012571,0.783323,0.927443,0.788935,2.733180,2.345882,0.585831,0.378117,0.564422,0.730071
2,0.531117,0.384819,0.402109,0.352820,0.300684,0.568432,0.270752,0.417030,0.216061,1.771576,0.392709,0.476015,0.329856,0.411426
3,1.188304,1.033520,1.280826,1.158938,1.233206,1.188980,1.098646,1.098297,2.350621,2.081692,0.679600,0.186661,0.626020,0.787213
4,0.298628,0.487721,0.722292,0.431413,0.963006,0.783956,0.741101,0.572742,0.123433,0.471346,0.430099,0.609716,0.533319,0.588045


In [172]:
trans_mat_df

,0,1,3,4,5,6,8,9,mean_trans_without_offer,mean_trans_with_offer,reward,gender,age,income
0,NaN,NaN,NaN,0.0,1.0,0.5,NaN,0.0,0.228500,0.2120,0.500000,NaN,NaN,NaN
1,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.689500,2.3220,0.500000,0.0,0.55,1.12
2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.238333,NaN,NaN,NaN,NaN,NaN
3,1.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,2.353200,2.0805,1.000000,0.0,0.75,1.00
4,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.155000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16989,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.286143,NaN,NaN,0.0,0.45,0.54
16990,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.325167,0.6460,0.500000,1.0,0.61,0.72
16991,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.496750,NaN,NaN,1.0,0.49,0.73
16992,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.412000,1.1450,0.833333,0.0,0.83,0.50
